In [3]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS


In [ ]:
from langchain_community.llms import HuggingFaceHub
import os

# Set your Hugging Face API Key
#os.environ["HUGGINGFACEHUB_API_TOKEN"] = 

# Initialize the Hugging Face model
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3", 
    model_kwargs={"temperature": 0.9, "max_length": 500}
)

# Generate a fancy restaurant name
name = llm.invoke("I want to open a restaurant for Mexican food. Suggest a fancy name for this.")

print(name)


C:\Users\saini\AppData\Local\Temp\ipykernel_20912\2742892928.py:8: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
c:\Users\saini\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\saini\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


I want to open a restaurant for Mexican food. Suggest a fancy name for this. Sure, I'd be happy to help brainstorm some ideas for a restaurant name that evokes a sense of luxury and ties into Mexican cuisine. Here are a few suggestions:

1. "Luminary Cazuela": Cazuela is a traditional Mexican cooking vessel, and luminary implies a prestigious or influential figure, which could suggest high-quality, authentic Mexican dishes.
2. "Saffron Oaxaca": Oaxaca is a state in Mexico known for its rich, complex flavors, and saffron is an expensive and luxurious spice. This name suggests a focus on gourmet Mexican dishes.
3. "Poblano Noble": Poblano is a type of chili pepper from Puebla, Mexico, and noble suggests sophistication and luxury.
4. "Mexico d'Oro": D'Oro is Italian for "of gold," and suggests luxury and opulence. Mexico d'Oro could evoke the idea of a golden, wealthy Mexico.
5. "Estrella de la Noche": Estrella means star in Spanish, and de la Noche means of the night. This name suggests 

### (1) Load data

In [22]:
from langchain.document_loaders import UnstructuredURLLoader

# Load documents from URLs
urls = [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
]
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

# 🔥 Ensure each document has a 'source' key in metadata
for doc, url in zip(data, urls):
    doc.metadata["source"] = url  # Manually add source metadata

print(data[0].metadata)  # Debug: Check if metadata exists


{'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}


### (2) Split data to create chunks

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [24]:
len(docs)

15

In [25]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_MARKETS_AS/MC_ENG_ROS_NWS_MKTS_AS_ATF_728\n\nGo PRO@₹1/dayPRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nOptions FestWebinar\n\nHomeNewsBusinessMarketsWall Street rises as Tesla soars on AI optimism\n\nTrending Topics\n\nSensex TodayIndusIn

### (3) Create embeddings for these chunks and save them to FAISS index

In [39]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# Create the embeddings of the chunks using openAIEmbeddings
embeddings =  HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

text_data = [doc.page_content for doc in docs]  # Extract text content

documents = [] 

for doc in docs:
    splits = text_splitter.split_text(doc.page_content)  # Split text into chunks
    for split in splits:
        documents.append({"text": split, "metadata": doc.metadata}) 
texts = [doc["text"] for doc in documents]
metadatas = [doc["metadata"] for doc in documents]        

vectorindex_huggingface = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

# Save and load FAISS index
vectorindex_huggingface.save_local("faiss_index")
new_vector_index = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [40]:
# Storing vector index create in local
file_path = "C:\\Users\\saini\\Downloads\\News Bot Research tool\\Notebook\\faiss_index\\index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_huggingface, f)

In [41]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

### (4) Retrieve similar embeddings for a given question and call LLM to retrieve final answer

In [42]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=HuggingFaceHub(client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.3', timeout=None)>, repo_id='mistralai/Mistral-7B-Instruct-v0.3', task='text-generation', model_kwargs={'temperature': 0.9, 'max_length': 500}), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Giv

In [43]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm",
      "question": "what is the price of Tiago iCNG?"
    },
    {
      "context": "T

c:\Users\saini\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:HuggingFaceHub] [5.92s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\nThe company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\nQuestion: what is the price of Tiago iCNG?\nRelevant text, if any: The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.",
        "generation_info": null,
 

c:\Users\saini\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:HuggingFaceHub] [1.83s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Given the following extracted parts of a long document and a question, create a final answer with references (\"SOURCES\"). \nIf you don't know the answer, just say that you don't know. Don't try to make up an answer.\nALWAYS return a \"SOURCES\" part in your answer.\n\nQUESTION: Which state/country's law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under 

{'answer': 'Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\n',
 'sources': "Which state/country's law governs the interpretation of the contract?"}

In [44]:
retrieved_docs = vectorindex_huggingface.similarity_search(query, k=5)
for doc in retrieved_docs:
    print(f"Document: {doc.page_content}\nMetadata: {doc.metadata}\n")


Document: The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.

The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.

Tata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up "appealing, holistic, and stronger than ever".

PTI

first published: Aug 4, 2023 02:17 pm
Metadata: {'source': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}

Document: Trending Topics

Indian RupeeMDR ChargesBusiness News LiveJoint Home LoanBank Holidays in March

Tata Motors launches Punch iCNG, price starts at Rs 7.1 lakh

The Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switched off at the time of refuelling and thermal incide

In [45]:
print(f"Number of documents in FAISS: {len(vectorindex_huggingface.docstore._dict)}")


Number of documents in FAISS: 36


In [46]:
retrieved_docs = vectorindex_huggingface.similarity_search(query, k=5)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

print(f"Context Passed to LLM:\n{context}\n")

response = llm.invoke(f"Answer the following based on the given context:\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:")
print(response)


Context Passed to LLM:
The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.

The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.

Tata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up "appealing, holistic, and stronger than ever".

PTI

first published: Aug 4, 2023 02:17 pm

Trending Topics

Indian RupeeMDR ChargesBusiness News LiveJoint Home LoanBank Holidays in March

Tata Motors launches Punch iCNG, price starts at Rs 7.1 lakh

The Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switched off at the time of refuelling and thermal incident protection that cuts off CNG supply to the engine and releases gas into the atmosphere, Tata Motors said in a statement.

PTI

PTI

Aug

c:\Users\saini\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


[llm/end] [llm:HuggingFaceHub] [3.52s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Answer the following based on the given context:\n\nContext:\nThe company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nTrending Topics\n\nIndian RupeeMDR ChargesBusiness News LiveJoint Home LoanBank Holidays in March\n\nTata Motors launches Punch iCNG, price starts at Rs 7.1 lakh\n\nThe Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switched off at th